In [7]:
import scanpy as sc
import numpy as np
import pandas as pd
import synapseclient

# Krishna 2021 dataset
Data source from: https://cellxgene.cziscience.com/collections/3f50314f-bdc9-40c6-8e4a-b0901ebfbe4c

Stored at: https://www.synapse.org/#!Synapse:syn54861463

In [8]:
syn = synapseclient.Synapse()
syn.login()

Welcome, heimann!



## Loading data

In [20]:
entity = syn.get('syn61363151')
adata = sc.read_h5ad(entity.path)
adata

AnnData object with n_obs × n_vars = 167283 × 15979
    obs: 'nGene', 'nUMI', 'author_type', 'author_sample', 'author_cluster', 'author_cluster_name', 'hca_data_portal_cellsuspension_uuid', 'hca_data_portal_donor_uuid', 'donor_id', 'suspension_type', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'
    uns: 'citation', 'default_embedding', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [11]:
adata.obs

,nGene,nUMI,author_type,author_sample,author_cluster,author_cluster_name,hca_data_portal_cellsuspension_uuid,hca_data_portal_donor_uuid,donor_id,suspension_type,...,tissue_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGTTGCC-1_UT1_Center,1534,2955,Tumor,UT1_Center,4,CD8A+ NK-like,5789095d-20f6-422d-a7b9-3fac82a7bf09,5e14a217-a126-426e-9f2b-c597d66d120f,Untreated 1,cell,...,tissue,"CD4-positive, alpha-beta T cell",10x 5' v2,clear cell renal carcinoma,Homo sapiens,female,kidney,unknown,69-year-old human stage,B|sp<rPuIF
AAACCTGAGCTGCCCA-1_UT1_Center,1679,3558,Tumor,UT1_Center,15,CD8A+ Proliferating,5789095d-20f6-422d-a7b9-3fac82a7bf09,5e14a217-a126-426e-9f2b-c597d66d120f,Untreated 1,cell,...,tissue,"CD4-positive, alpha-beta T cell",10x 5' v2,clear cell renal carcinoma,Homo sapiens,female,kidney,unknown,69-year-old human stage,iYc)udY*MZ
AAACCTGAGTGGACGT-1_UT1_Center,1374,2539,Tumor,UT1_Center,19,CD8A+ Exhausted IEG,5789095d-20f6-422d-a7b9-3fac82a7bf09,5e14a217-a126-426e-9f2b-c597d66d120f,Untreated 1,cell,...,tissue,"naive thymus-derived CD8-positive, alpha-beta ...",10x 5' v2,clear cell renal carcinoma,Homo sapiens,female,kidney,unknown,69-year-old human stage,!2Uv>ugM<D
AAACCTGCAGGTCGTC-1_UT1_Center,1036,1636,Tumor,UT1_Center,10,Conventional NK,5789095d-20f6-422d-a7b9-3fac82a7bf09,5e14a217-a126-426e-9f2b-c597d66d120f,Untreated 1,cell,...,tissue,mature NK T cell,10x 5' v2,clear cell renal carcinoma,Homo sapiens,female,kidney,unknown,69-year-old human stage,YxEMb(IMgi
AAACCTGCAGGTCTCG-1_UT1_Center,912,1752,Tumor,UT1_Center,2,CD8A+ Tissue-resident,5789095d-20f6-422d-a7b9-3fac82a7bf09,5e14a217-a126-426e-9f2b-c597d66d120f,Untreated 1,cell,...,tissue,"CD4-positive, alpha-beta T cell",10x 5' v2,clear cell renal carcinoma,Homo sapiens,female,kidney,unknown,69-year-old human stage,qr5_$eW?KN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCCATGAAC-1_t4_PBMC,589,1028,PBMC,t4_PBMC,3,CD4+ Naive,c4fabc37-113d-4bd5-a930-2bebd4a594fa,c9c7acc9-c835-4d88-97cb-0f76af3133cb,Ipi/Nivo complete response,cell,...,tissue,naive T cell,10x 5' v2,clear cell renal carcinoma,Homo sapiens,male,blood,unknown,63-year-old human stage,7hY{4EU1~%
TTTGTCATCGCGGATC-1_t4_PBMC,632,1092,PBMC,t4_PBMC,10,Conventional NK,c4fabc37-113d-4bd5-a930-2bebd4a594fa,c9c7acc9-c835-4d88-97cb-0f76af3133cb,Ipi/Nivo complete response,cell,...,tissue,mature NK T cell,10x 5' v2,clear cell renal carcinoma,Homo sapiens,male,blood,unknown,63-year-old human stage,Cv5~7E&Px>
TTTGTCATCTCAAACG-1_t4_PBMC,651,1108,PBMC,t4_PBMC,3,CD4+ Naive,c4fabc37-113d-4bd5-a930-2bebd4a594fa,c9c7acc9-c835-4d88-97cb-0f76af3133cb,Ipi/Nivo complete response,cell,...,tissue,naive T cell,10x 5' v2,clear cell renal carcinoma,Homo sapiens,male,blood,unknown,63-year-old human stage,t)0urojf(*


In [13]:
adata.var

,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length
ENSG00000228463,False,ENSG00000228463.10,NCBITaxon:9606,gene,8224
ENSG00000225880,False,LINC00115,NCBITaxon:9606,gene,3312
ENSG00000230368,False,FAM41C,NCBITaxon:9606,gene,1971
ENSG00000187634,False,SAMD11,NCBITaxon:9606,gene,4172
ENSG00000188976,False,NOC2L,NCBITaxon:9606,gene,5540
...,...,...,...,...,...
ENSG00000160298,False,C21orf58,NCBITaxon:9606,gene,5315
ENSG00000160299,False,PCNT,NCBITaxon:9606,gene,17507
ENSG00000160305,False,DIP2A,NCBITaxon:9606,gene,13758
ENSG00000160307,False,S100B,NCBITaxon:9606,gene,1806


In [7]:
#The authors also made available a file with cell annotations
entity = syn.get('syn59205039')
cell_annot = pd.read_csv(entity.path, sep='\t')
cell_annot.head()

,cell,type,region,Sample,Sample2,cluster,cluster_name,UMAP1,UMAP2,Sample_name
0,AAACCTGAGCGTTGCC-1_UT1_Center,Tumor,Center,UT1,UT1_Center,4,CD8A+ NK-like,5.896925,-4.373115,Untreated 1
1,AAACCTGAGCTGCCCA-1_UT1_Center,Tumor,Center,UT1,UT1_Center,15,CD8A+ Proliferating,3.004102,1.834490,Untreated 1
2,AAACCTGAGTGGACGT-1_UT1_Center,Tumor,Center,UT1,UT1_Center,19,CD8A+ Exhausted IEG,1.148642,1.801541,Untreated 1
3,AAACCTGCAGGTCGTC-1_UT1_Center,Tumor,Center,UT1,UT1_Center,10,Conventional NK,3.649340,-8.709116,Untreated 1
4,AAACCTGCAGGTCTCG-1_UT1_Center,Tumor,Center,UT1,UT1_Center,2,CD8A+ Tissue-resident,4.961056,-0.395439,Untreated 1


In [8]:
#Inspect more closely unique values for each column
pd.unique(cell_annot["cluster_name"])

array(['CD8A+ NK-like', 'CD8A+ Proliferating', 'CD8A+ Exhausted IEG',
       'Conventional NK', 'CD8A+ Tissue-resident',
       'CD45- Vascular Endothelium', 'NK HSP+', 'CD4+ Activated IEG',
       'TAM/TCR (Ambiguos)', 'CD45- PAX8+ renal epithelium', 'TAM HLAhi',
       'CD8A+ Exhausted', 'CD14+ Monocyte', 'CD14+/CD16+ Monocyte',
       'CD45- Myofibroblast', 'CD4+ Naive', 'cDC2', 'TAM HLAint',
       'Ambiguous', 'Mast', 'CD4+ Proliferating', 'CD45- ccRCC CA9+',
       'CD4+ Effector', 'TAM ISGint', 'B cell', 'CD4+ Treg', 'TAM ISGhi',
       'cDC1', 'Megakaryocyte', 'pDC', 'Ambiguous/Dead'], dtype=object)

# QC

According to the study manuscript, the following filtering was performed:

*the count matrix of cell barcodes by genes used for downstream analysis was generated using the Cell Ranger count function with parameter –expect-cells = 3000. The raw count matrix for each of the 29 samples was obtained from the Cell Ranger count filter_matrix output. Briefly, for each sample, Cell Ranger plots the total UMI count against the cell barcode rank in decreasing order of total counts and filters cell barcodes out of the resulting count matrix based on the inflection point of the plot. This step minimizes the number of empty droplets that are included in downstream analyses. We created an initial count matrix combining all 29 samples from all patients. Following count matrix generation, cells with >20% of transcripts derived from mitochondrial genes were considered apoptotic and were thus excluded. Following this step, all mitochondrial genes were filtered out of the count matrix. Ribosomal genes and the noncoding RNAs NEAT1 and MALAT1 were excluded due to prior reports of strong influence on downstream clustering (Freytag et al., 2018). Genes with mean raw count <3.0 were removed from
the analysis, resulting in a count matrix of 167283 cells and 16,323 genes for downstream analysis.*

However, there is no record of these metrics in the object. We only know that the file went through preprocessing because the number of cells the described in the manuscript. The number of genes in the file is smaller than reported in the manuscript, which suggested that it was submitted to further filtering. We will not make any additional filtering at this moment.

# Formatting to run pseudobulk

In [21]:
#changing column names to gene symbols
# #change gene id from ensembl to gene symbol (stored in var)
adata.var = adata.var.reset_index().rename(columns={'gene_ids': 'ensembl_gene_id'}
                                          ).set_index('feature_name')


#let's take a look at the raw counts
adata.layers['counts'] = adata.X
adata.to_df(layer = "counts")

[WARNING] /usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:767: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['ENSG00000228463.10', 'LINC00115', 'FAM41C', 'SAMD11', 'NOC2L']

    Inferred to be: categorical

  value_idx = self._prep_dim_index(value.index, attr)



feature_name,ENSG00000228463.10,LINC00115,FAM41C,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,...,LSS,MCM3AP-AS1,MCM3AP,ENSG00000239415.1,YBEY,C21orf58,PCNT,DIP2A,S100B,PRMT2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGTTGCC-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.477983,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
AAACCTGAGCTGCCCA-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.337778
AAACCTGAGTGGACGT-1_UT1_Center,0.0,0.0,0.0,0.0,1.597073,0.0,0.0,0.0,0.0,1.597073,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
AAACCTGCAGGTCGTC-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.961849,0.0,0.000000
AAACCTGCAGGTCTCG-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCCATGAAC-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.372822,0.0,0.000000
TTTGTCATCGCGGATC-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.960883,...,0.0,0.0,2.318213,0.0,0.0,0.0,0.0,0.000000,0.0,2.960883
TTTGTCATCTCAAACG-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [22]:
# Normalize and log-transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers['normalized'] = adata.X

In [23]:
adata.to_df(layer="normalized")

feature_name,ENSG00000228463.10,LINC00115,FAM41C,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,...,LSS,MCM3AP-AS1,MCM3AP,ENSG00000239415.1,YBEY,C21orf58,PCNT,DIP2A,S100B,PRMT2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGTTGCC-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.891571,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
AAACCTGAGCTGCCCA-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.789586
AAACCTGAGTGGACGT-1_UT1_Center,0.0,0.0,0.0,0.0,1.984895,0.0,0.0,0.0,0.0,1.984895,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
AAACCTGCAGGTCGTC-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.298865,0.0,0.000000
AAACCTGCAGGTCTCG-1_UT1_Center,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCCATGAAC-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.815600,0.0,0.000000
TTTGTCATCGCGGATC-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.987717,...,0.0,0.0,2.756903,0.0,0.0,0.0,0.0,0.000000,0.0,2.987717
TTTGTCATCTCAAACG-1_t4_PBMC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [32]:
#This dataset has different names for cells than what we will use in iAtlas, so let's update them
def func(a):
    if a in ["CD4+ Treg", "CD4+ Naive", "CD4+ Effector", "CD4+ Activated IEG", "CD4+ Proliferating", "CD8A+ Proliferating", "CD8A+ Exhausted IEG", "CD8A+ Exhausted", "CD8A+ NK-like", "CD8A+ Tissue-resident"]:
        return "T cell"
    elif a in ["TAM HLAhi", "TAM HLAint", "TAM ISGhi", "TAM ISGint"]:
        return "macrophage"
    elif a in "B cell":
        return "B cell"
    elif a in "Mast":
        return "mast cell"
    elif a in "Megakaryocyte":
        return "megakaryocyte"
    elif a in "CD45- Vascular Endothelium":
        return "endothelium"
    elif a in "CD45- PAX8+ renal epithelium":
        return "endothelium"
    elif a in "CD45- Myofibroblast":
        return "fibroblast"
    elif a in "CD45- ccRCC CA9+":
        return "tumor"
    elif a in ["CD14+ Monocyte", "CD14+/CD16+ Monocyte"]:
        return "monocyte"
    elif a in ["Conventional NK", "NK HSP+"]:
        return "NK"
    elif a in ["cDC1", "cDC2", "pDC"]:
        return "Dendritic cell"
    else:
        return a

adata.obs["cell_type_iatlas"] = adata.obs["author_cluster_name"].apply(lambda x: func(x))

In [33]:
adata.obs["cell_type_iatlas"].value_counts()

cell_type_iatlas
T cell                73806
macrophage            20547
endothelium           14062
monocyte              14011
NK                    12538
Ambiguous             10924
Dendritic cell         6665
tumor                  5715
fibroblast             3076
TAM/TCR (Ambiguos)     2797
B cell                 2722
mast cell               242
megakaryocyte           117
Ambiguous/Dead           61
Name: count, dtype: int64

In [34]:
adata.write('krishna_iatlas_from_h5adfile.h5ad')
file_entity = synapseclient.File('krishna_iatlas_from_h5adfile.h5ad', 'syn59202660')
file_entity = syn.store(file_entity)

Uploading to Synapse storage: 100%|██████████| 7.82G/7.82G [03:15<00:00, 40.0MB/s, krishna_iatlas_from_h5adfile.h5ad]
